In [26]:
import csv
from datetime import datetime

# HL7 Template Fields (MSH, PID, OBR, OBX)
def generate_msh(segment_date, message_control_id):
    return f"MSH|^~\\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|{segment_date}||ORU^R01|{message_control_id}|P|2.5.1"

def generate_pid(patient_id, last_name, first_name, middle_initial, dob, sex, address):
    return f"PID|||{patient_id}^^^^TWSMAMRN||{last_name}^{first_name}^{middle_initial}||{dob}|{sex}|||{address}"

def generate_obr(observation_time):
    return f"OBR|1||||||{observation_time}"

def generate_obx(index, loinc_code, description, result, unit):
    return f"OBX|{index}||{loinc_code}^{description}^LN||{result}|{unit}"

# Mapping of metric descriptions to LOINC codes and descriptions
loinc_mapping = {
    "Heart Rate": ("8867-4", "PUL RATE"),
    "Systolic": ("8480-6", "BP SYS"),
    "Diastolic": ("8462-4", "BP DIAS"),
    "Weight": ("3141-9", "WGT"),
    "Height": ("8302-2", "HGT"),
    "O2 Saturation": ("59408-5", "O2"),
    "Temperature": ("8310-5", "TEMP"),
}

# Adjusting the script based on the updated format and rules
def create_hl7_from_csv_with_rules(input_file_path, output_file_path):
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        csv_reader = csv.reader(infile, delimiter='|')
        current_patient = None
        current_performed_dttm = None
        index = 1
        
        for row in csv_reader:
            # Extract relevant fields based on their positions in the row
            patient_id = row[0]
            visit_id = row[1]
            last_name = row[2]
            first_name = row[3]
            middle_initial = row[4]
            dob = row[5].replace("-", "")
            sex = row[6]
            address = f"{row[30]}^^{row[32]}^{row[33]}^{row[34]}"
            metric_desc = row[7]
            result = row[9]
            unit = row[19]
            performed_dttm = row[12]
            observation_time = performed_dttm.replace("-", "").replace(":", "").replace(" ", "")
            performed_dttm_short = performed_dttm.split(" ")[0].replace("-", "")
            
            # Create Message Control ID using VisitID_PatientID_PerformedDTTM
            message_control_id = f"{visit_id}_{patient_id}_{performed_dttm_short}"
            
            # Check if we need to start a new encounter
            if current_performed_dttm != performed_dttm:
                if current_performed_dttm:  # Close the previous message with a newline
                    outfile.write("\n\n")
                
                # Write MSH segment
                segment_date = datetime.now().strftime("%Y%m%d%H%M%S")
                msh = generate_msh(segment_date, message_control_id)
                outfile.write(f"{msh}\n")
                
                # Write PID segment
                pid = generate_pid(patient_id, last_name, first_name, middle_initial, dob, sex, address)
                outfile.write(f"{pid}\n")
                
                # Write OBR segment
                obr = generate_obr(observation_time[:14])
                outfile.write(f"{obr}\n")
                
                # Reset OBX index and update current_performed_dttm
                index = 1
                current_performed_dttm = performed_dttm
            
            # Map the metric description to the correct LOINC code and description
            loinc_lab_code, loinc_desc = loinc_mapping.get(metric_desc, (None, None))
            if loinc_lab_code:
                # Write OBX segment for each metric
                obx = generate_obx(index, loinc_lab_code, loinc_desc, result, unit)
                outfile.write(f"{obx}\n")
                index += 1

# Define file paths
input_file_path = 'Input.txt'  # Replace with the actual path to the input file
output_file_path = 'output_hl7_with_rules.txt'

# Run the updated HL7 generation process
create_hl7_from_csv_with_rules(input_file_path, output_file_path)

# Load the generated output to check the results
with open(output_file_path, 'r') as file:
    hl7_output_rules = file.read()

# Display the first 2000 characters of the generated HL7 output for validation
print(hl7_output_rules[:2000])


MSH|^~\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|20241021005426||ORU^R01|20004200_420_20211022|P|2.5.1
PID|||420^^^^TWSMAMRN||Martinez^David^E||20080419|M|||Martinez^^4479 E Mesquite Desert Trl^^Tucson
OBR|1||||||20211022170300
OBX|1||8867-4^PUL RATE^LN||68|bpm
OBX|2||8480-6^BP SYS^LN||110|mm Hg
OBX|3||8462-4^BP DIAS^LN||62|mm Hg
OBX|4||8302-2^HGT^LN||156.21|in
OBX|5||3141-9^WGT^LN||39.26|lb
OBX|6||59408-5^O2^LN||97|
OBX|7||8310-5^TEMP^LN||36.94|F


MSH|^~\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|20241021005426||ORU^R01|20004200_420_20211110|P|2.5.1
PID|||420^^^^TWSMAMRN||Martinez^David^E||20080419|M|||Martinez^^4479 E Mesquite Desert Trl^^Tucson
OBR|1||||||20211110131100
OBX|1||8867-4^PUL RATE^LN||68|bpm
OBX|2||8310-5^TEMP^LN||36.56|F
OBX|3||8480-6^BP SYS^LN||114|mm Hg
OBX|4||8462-4^BP DIAS^LN||68|mm Hg
OBX|5||3141-9^WGT^LN||37.82|lb
OBX|6||8302-2^HGT^LN||156.21|in
OBX|7||59408-5^O2^LN||98|


MSH|^~\&|TouchWorks|Southwest Medica

In [30]:
import csv
from datetime import datetime

# HL7 Template Fields (MSH, PID, OBR, OBX)
def generate_msh(segment_date, message_control_id):
    return f"MSH|^~\\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|{segment_date}||ORU^R01|{message_control_id}|P|2.5.1"

def generate_pid(patient_id, last_name, first_name, middle_initial, dob, sex, address):
    return f"PID|||{patient_id}^^^^TWSMAMRN||{last_name}^{first_name}^{middle_initial}||{dob}|{sex}|||{address}"

def generate_obr(observation_time):
    return f"OBR|1||||||{observation_time}"

def generate_obx(index, loinc_code, description, result, unit):
    return f"OBX|{index}||{loinc_code}^{description}^LN||{result}|{unit}"

# Mapping of metric descriptions to LOINC codes and descriptions
loinc_mapping = {
    "Heart Rate": ("8867-4", "PUL RATE"),
    "Systolic": ("8480-6", "BP SYS"),
    "Diastolic": ("8462-4", "BP DIAS"),
    "Weight": ("3141-9", "WGT"),
    "Height": ("8302-2", "HGT"),
    "O2 Saturation": ("59408-5", "O2"),
    "Temperature": ("8310-5", "TEMP"),
}

def create_hl7_from_csv_with_rules(input_file_path, output_file_path):
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        csv_reader = csv.reader(infile, delimiter='|')
        current_patient = None
        current_performed_dttm = None
        index = 1
        
        for row in csv_reader:
            # Extract relevant fields based on their positions in the row
            patient_id = row[1]  # Use row[1] instead of row[0] to fix Patient ID
            visit_id = row[0]  # Now row[0] has Visit ID
            last_name = row[2]
            first_name = row[3]
            middle_initial = row[4]
            dob = row[5].replace("-", "")
            sex = row[6]
            address = f"{row[30]}^^{row[32]}^{row[33]}^{row[34]}^AZ^857063013"
            metric_desc = row[7]
            result = row[9]
            unit = row[19]
            performed_dttm = row[12]
            observation_time = performed_dttm.replace("-", "").replace(":", "").replace(" ", "")
            performed_dttm_short = performed_dttm.split(" ")[0].replace("-", "")
            
            # Create Message Control ID using VisitID_PatientID_PerformedDTTM
            message_control_id = f"{visit_id}_{patient_id}_{performed_dttm_short}"
            
            # Check if we need to start a new encounter
            if current_performed_dttm != performed_dttm:
                if current_performed_dttm:  # Close the previous message with a newline
                    outfile.write("\n\n")
                
                # Write MSH segment
                segment_date = datetime.now().strftime("%Y%m%d%H%M%S")
                msh = generate_msh(segment_date, message_control_id)
                outfile.write(f"{msh}\n")
                
                # Write PID segment
                pid = generate_pid(patient_id, last_name, first_name, middle_initial, dob, sex, address)
                outfile.write(f"{pid}\n")
                
                # Write OBR segment
                obr = generate_obr(observation_time[:14])
                outfile.write(f"{obr}\n")
                
                # Reset OBX index and update current_performed_dttm
                index = 1
                current_performed_dttm = performed_dttm
            
            # Map the metric description to the correct LOINC code and description
            loinc_lab_code, loinc_desc = loinc_mapping.get(metric_desc, (None, None))
            if loinc_lab_code:
                # Write OBX segment for each metric
                obx = generate_obx(index, loinc_lab_code, loinc_desc, result, unit)
                outfile.write(f"{obx}\n")
                index += 1

# Define file paths
input_file_path = 'Input.txt'  # Replace with the actual path to the input file
output_file_path = 'output_hl7_with_rules.txt'

# Run the updated HL7 generation process
create_hl7_from_csv_with_rules(input_file_path, output_file_path)

# Load the generated output to check the results
with open(output_file_path, 'r') as file:
    hl7_output_rules = file.read()

# Display the first 2000 characters of the generated HL7 output for validation
print(hl7_output_rules[:2000])


MSH|^~\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|20241021010656||ORU^R01|420_20004200_20211022|P|2.5.1
PID|||20004200^^^^TWSMAMRN||Martinez^David^E||20080419|M|||Martinez^^4479 E Mesquite Desert Trl^^Tucson^AZ^857063013
OBR|1||||||20211022170300
OBX|1||8867-4^PUL RATE^LN||68|bpm
OBX|2||8480-6^BP SYS^LN||110|mm Hg
OBX|3||8462-4^BP DIAS^LN||62|mm Hg
OBX|4||8302-2^HGT^LN||156.21|in
OBX|5||3141-9^WGT^LN||39.26|lb
OBX|6||59408-5^O2^LN||97|
OBX|7||8310-5^TEMP^LN||36.94|F


MSH|^~\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|20241021010656||ORU^R01|420_20004200_20211110|P|2.5.1
PID|||20004200^^^^TWSMAMRN||Martinez^David^E||20080419|M|||Martinez^^4479 E Mesquite Desert Trl^^Tucson^AZ^857063013
OBR|1||||||20211110131100
OBX|1||8867-4^PUL RATE^LN||68|bpm
OBX|2||8310-5^TEMP^LN||36.56|F
OBX|3||8480-6^BP SYS^LN||114|mm Hg
OBX|4||8462-4^BP DIAS^LN||68|mm Hg
OBX|5||3141-9^WGT^LN||37.82|lb
OBX|6||8302-2^HGT^LN||156.21|in
OBX|7||59408-5^O2^LN||98|




In [33]:
import csv
from datetime import datetime

# Function to generate the MSH segment
def generate_msh(segment_date, message_control_id):
    return f"MSH|^~\\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|{segment_date}||ORU^R01|{message_control_id}|P|2.5.1"

# Function to generate the PID segment
def generate_pid(patient_id, last_name, first_name, middle_initial, dob, sex, address):
    return f"PID|||{patient_id}^^^^TWSMAMRN||{last_name}^{first_name}^{middle_initial}||{dob}|{sex}|||{address}"

# Function to generate the OBR segment
def generate_obr(observation_time):
    return f"OBR|1||||||{observation_time}"

# Function to generate the OBX segment
def generate_obx(index, loinc_code, description, result, unit):
    return f"OBX|{index}||{loinc_code}^{description}^LN||{result}|{unit}"

# Mapping of metric descriptions to LOINC codes and descriptions
loinc_mapping = {
    "Heart Rate": ("8867-4", "PUL RATE"),
    "Systolic": ("8480-6", "BP SYS"),
    "Diastolic": ("8462-4", "BP DIAS"),
    "Weight": ("3141-9", "WGT"),
    "Height": ("8302-2", "HGT"),
    "O2 Saturation": ("59408-5", "O2"),
    "Temperature": ("8310-5", "TEMP"),
}

# Unit conversion functions
def kg_to_lb(kg):
    return round(float(kg) * 2.20462, 2)

def cm_to_in(cm):
    return round(float(cm) / 2.54, 2)

def celsius_to_fahrenheit(c):
    return round((float(c) * 9/5) + 32, 2)

# Function to create HL7 messages
def create_hl7_from_input(input_file_path, output_file_path):
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        csv_reader = csv.reader(infile, delimiter='|')
        current_performed_dttm = None
        index = 1
        
        for row in csv_reader:
            # Map fields from input data
            visit_id = row[0]
            patient_id = row[1]
            last_name = row[2]
            first_name = row[3]
            middle_initial = row[4]
            dob = row[5].replace("-", "")
            sex = row[6]
            address = f"{row[30]}^^{row[32]}^{row[33]}^{row[34]}^AZ^{row[35]}"
            metric_desc = row[7]
            result = row[9]
            unit = row[19]
            performed_dttm = row[12]
            observation_time = performed_dttm.replace("-", "").replace(":", "").replace(" ", "")
            performed_dttm_short = performed_dttm.split(" ")[0].replace("-", "")
            
            # Generate Message Control ID
            message_control_id = f"{visit_id}_{patient_id}_{performed_dttm_short}"
            
            # Generate MSH, PID, OBR
            if current_performed_dttm != performed_dttm:
                if current_performed_dttm:
                    outfile.write("\n\n")
                segment_date = datetime.now().strftime("%Y%m%d%H%M%S")
                msh = generate_msh(segment_date, message_control_id)
                pid = generate_pid(patient_id, last_name, first_name, middle_initial, dob, sex, address)
                obr = generate_obr(observation_time[:14])
                outfile.write(f"{msh}\n{pid}\n{obr}\n")
                current_performed_dttm = performed_dttm
                index = 1
            
            # Generate OBX for each metric
            loinc_lab_code, loinc_desc = loinc_mapping.get(metric_desc, (None, None))
            if loinc_lab_code:
                if metric_desc == "Weight":
                    result = kg_to_lb(result)
                    unit = "lb"
                elif metric_desc == "Height":
                    result = cm_to_in(result)
                    unit = "in"
                elif metric_desc == "Temperature":
                    result = celsius_to_fahrenheit(result)
                    unit = "F"
                
                obx = generate_obx(index, loinc_lab_code, loinc_desc, result, unit)
                outfile.write(f"{obx}\n")
                index += 1

# Define file paths
input_file_path = 'Input.txt'  # Path to your input file
output_file_path = 'output_hl7.txt'

# Run the HL7 generation
create_hl7_from_input(input_file_path, output_file_path)

# Output the generated HL7 for review
with open(output_file_path, 'r') as file:
    hl7_output = file.read()
print(hl7_output)


MSH|^~\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|20241021012813||ORU^R01|420_20004200_20211022|P|2.5.1
PID|||20004200^^^^TWSMAMRN||Martinez^David^E||20080419|M|||Martinez^^4479 E Mesquite Desert Trl^^Tucson^AZ^AZ
OBR|1||||||20211022170300
OBX|1||8867-4^PUL RATE^LN||68|bpm
OBX|2||8480-6^BP SYS^LN||110|mm Hg
OBX|3||8462-4^BP DIAS^LN||62|mm Hg
OBX|4||8302-2^HGT^LN||61.5|in
OBX|5||3141-9^WGT^LN||86.55|lb
OBX|6||59408-5^O2^LN||97|
OBX|7||8310-5^TEMP^LN||98.49|F


MSH|^~\&|TouchWorks|Southwest Medical Associates|Rhapsody^Rhapsody|Epic^Epic|20241021012813||ORU^R01|420_20004200_20211110|P|2.5.1
PID|||20004200^^^^TWSMAMRN||Martinez^David^E||20080419|M|||Martinez^^4479 E Mesquite Desert Trl^^Tucson^AZ^AZ
OBR|1||||||20211110131100
OBX|1||8867-4^PUL RATE^LN||68|bpm
OBX|2||8310-5^TEMP^LN||97.81|F
OBX|3||8480-6^BP SYS^LN||114|mm Hg
OBX|4||8462-4^BP DIAS^LN||68|mm Hg
OBX|5||3141-9^WGT^LN||83.38|lb
OBX|6||8302-2^HGT^LN||61.5|in
OBX|7||59408-5^O2^LN||98|


MSH|^~\&|TouchWork